# Environment Setup: 機械学習モデルの開発環境を構築する

## なにをするのか

**「データ分析」に必要な「環境の仕様」と構築が「合意」されていること**がEnvironment Setupのゴールです。鍵かっこのついた3つのポイントを解説します。

1. データ分析
   * ビジネスゴールを達成するために今あるデータが必要十分か検証証するプロセス(Analyzeフェーズ)。
2. 環境の仕様
   * 開発環境の構成図、Pythonやライブラリのバージョンなど。
3. 合意
   * MLOpsチーム内だけでなく、DevOpsチームやIT監督者といった機械学習モデルのリリースに必要な関係者の合意。

ゴールに至るため、**開発環境の構築手順書を作成し、文書で開発環境について合意します。**

手順書を作成するのは、MLOpsチームが手順書を参照し環境を再現する、DevOpsチームがプロダクトと連携させる方法を検討できるようにするためです。手順書はGitHubなどバージョン管理できるプラットフォーム上に置くのが好ましいです。[Wantedly Engineering Handbook](https://docs.wantedly.dev/fields/dev-tools/codespaces)は環境構築手順書の素晴らしい例です。

## なぜやるのか

プロダクション環境へのリリースを見据えて開発するためです。

DevOpsチームはただでさえソフトウェア開発で忙しいので、リソースを割いて機械学習モデルを組み込むのは大変です。**事前に機械学習モデルの開発環境について合意しておきリリース時の驚きを減らす**ことが、負担の軽減につながります。ひいては、プロダクション環境で機械学習モデルが動く確率を高められます。

プロダクションで推論しない機械学習モデルは車に搭載されていないエンジンみたいなもので、性能が良くても価値を生むことはありません。

## どうやるのか

1. プログラミング演習
   * Studio Labを使いCondaで開発環境を構築し、構築手順を共有する方法を学びます。
2. コミュニケーション演習
   * 開発環境を構築する際に行うべきコミュニケーションを学びます。

オンプレミスやクラウドでJupyterLabを起動する環境を作るにはネットワークや権限の設計、構築が必要です。今回はそうした構築を行わず無料の環境でまず試すというシーンを想定しSageMaker Studio Labを使用します。

開発環境の構築には[Conda](https://docs.conda.io/en/latest/)を使用します。Condaはプロジェクトごとの開発環境(仮想環境)を作成、管理するためのオープンソースです。なおPython以外の言語の環境も構築できます。

**Condaとvirtualenv(venv)・pip・pipenv・poetryとの差異**
[virtualenv](https://virtualenv.pypa.io/en/latest/)([venv](https://docs.python.org/ja/3/library/venv.html))・[pipenv](https://github.com/pypa/pipenv)・[poetry](https://github.com/python-poetry/poetry)について、仮想環境が作れる点は同じです。CondaはシステムにインストールされているPythonと別個にPythonをインストールして実行環境が作成できる点が異なります。[pyenv](https://github.com/pyenv/pyenv)と同じ機能を持っています。

pipenv・poetry・pipとはパッケージのインストールができる点は同じです。CondaはPythonパッケージ以外にPython本体やPythonとは関係がないライブラリもインストールすることができます。(そのため)PyPIからではなくanaconda.orgからインストールする点が異なります。ただCondaで作成した環境でpipを利用することは可能です。同じパッケージをCondaとpip両方でインストールしないよう注意してください。

| ツール名 | 開発 | 特徴・歴史的経緯 |
|---|---|---|
| virtualenv | PyPA | もっと歴史が長い?仮想環境の構築ツール。 |
| venv | PyPA | virtualenvの機能が一部切り出され、Python   3.3から標準で付属。 |
| pip | PyPA | Python公式のパッケージ管理ツール。Python   3.4から標準で付属。 |
| pipenv | PyPA | 仮想環境の作成はvirtualenv、ライブラリのインストールはpip、と使い分けるのではなく一つのツールで済むよう開発されている。 |
| Conda | Anaconda,   Inc. | Pythonパッケージが集まるリポジトリであるPyPIではなく、anaconda.orgからWindowsやLinuxなど環境に応じたバイナリをインストールする。        PythonのパッケージはPython以外のライブラリ(HDF5やMKL、LLVMなど)に依存するにもかかわらずpipでインストールできないためインストールのビルド時にエラーになることがしばしばあった。コンパイル済みのバイナリ、Python以外のライブラリインストールできるようにしたもの(※Wheelが登場しpipでもバイナリがインストールできるようになった)。 |
| poetry | Sébastien   Eustace(python-poetry) | PyPIへのパッケージ公開までサポートする。Pythonのプロジェクト(パッケージ)情報をpyproject.toml一本にまとめて管理する方針。PyPIのJSON   APIが使えれば依存関係解決に使用するため、手元にダウンロードしてbuildするpipenvよりもlockが高速。 |

CondaはいくつかのPythonのパッケージと一体となった配布形式があります。Anacondaは代表的な配布形式の1つであり、商用利用する際は有償になります。詳細は[利用規約](https://www.anaconda.com/terms-of-service)を参照してください。Studio LabではCondaを使用していますがAnacondaに含まれるPythonパッケージを利用していないため無償で利用できます。詳細は[GitHubのIssue](https://github.com/aws/studio-lab-examples/issues/100)を参照してください。

### プログラミング演習

環境の構築手順は次の通りです。

1. SageMaker Studio Labを起動
2. `git clone`でファイルを取得
3. Condaを使用しPythonとライブラリをインストールする
4. インストールしたライブラリを記録したファイルを作成する

![00_environment_setup_001.PNG](./images/00_environment_setup_001.PNG)


#### 1. SageMaker Studio Labを起動

Amazon SageMaker Studio Labはメールアドレスのみで、無料で利用できるJupyter Lab環境です。次のような特色があります。

* 12時間のCPU、4時間のGPUが利用可能。
   * 超過した場合セッションが切断されランタイムを再度起動する必要がありますが、ファイルは静的ストレージに維持されます。
* 15GBの静的ストレージがあり、作業内容が保管される。
* Gitと統合されており、GUIでファイルのバージョン管理が可能。
* Notebookに加えてTerminalも利用可能。

利用方法については、[Amazon SageMaker Studio Lab の使い方](https://github.com/aws-studiolab-jp/awesome-studio-lab-jp/blob/main/README_usage.md)を参照してください。ログインした後、CPU/GPUいずれかのインスタンスを選択し起動します。

![00_environment_setup_002.PNG](./images/00_environment_setup_002.PNG)

##### JupyterLabをローカルで構築する場合

Studio Labが利用できない場合、次の手順でJupyterLab環境が構築できます。

1. [Minicondaのインストール](https://docs.conda.io/en/latest/miniconda.html)
2. Terminalから`ml-handson`環境を作成: `conda create -n ml-handson python jupyterlab pandas scikit-learn matplotlib ipython ipykernel jinja2`
3. Terminalから`ml-handson`環境を有効化: `conda activate ml-handson`
4. TerminalからJupyter Labを起動: `jupyter lab`

#### 2. `git clone`でファイルを取得

ハンズオンのコンテンツをダウンロードするために、Gitを使用します(Gitの説明は後ほど行います)。まずTerminalを起動します。

![00_environment_setup_003.PNG](images/00_environment_setup_003.PNG)

次のコマンドを実行しましょう。

```
git clone https://github.com/aws-samples/aws-ml-enablement-handson.git
```

ファイルエクスプローラーから`aws-ml-enablement-handson`というフォルダが見えたら成功です。

##### フォルダ構成

本ハンズオンのディレクトリ構造は次のようになっています。

```
├── LICENSE
├── README.md          <- リポジトリの概要を記載した文書
├── environment.yml    <- ハンズオンの動作に必要なライブラリをまとめたファイル
├── data
│   ├── raw            <- 生データ
│   ├── interim        <- 中間データ
│   └── processed      <- 前処理済みデータ
│
├── docs               <- ドキュメント
│
├── models             <- 学習済みモデル、前処理用パイプライン
│
└── notebooks          <- Jupyter notebooks
```

ディレクトリの構成は、[cookiecutter-data-science](https://github.com/drivendata/cookiecutter-data-science)というテンプレートに沿っています。

##### Gitについて

ファイルの管理にGitを使用しています。Gitでは`commit`が変更を記録する単位です、1回の`commit`で記録する内容は、`add`で登録を行います。`commit`は`git commit`というコマンド、`add`は`git add`というコマンドで行います(Studio Labでは、GUIで操作ができます)。Gitの使い方に不案内な場合は、次の記事を参照してください。

[データサイエンティストのための Git 入門](https://aws.amazon.com/jp/builders-flash/202207/git-introduction-for-data-schientist/?awsf.filter-name=*all)

一般的に、変更を加える際は本体のコードとは別に`branch`を作成しておき作業します。`branch`での作業が終了したら、本体に取込みます。この作業を`merge`と呼びます。本体がGitHubのようなリモートリポジトリにあれば、`git clone`で手元の端末にファイルを取得し、`git push`で`commit`した変更をリモートリポジトリ/ブランチに反映します。`merge`の操作はGitHub上で行えます。

![00_environment_setup_004.PNG](./images/00_environment_setup_004.PNG)

ブランチの作成方法やコミットの単位は、既存の開発ルールに倣うのが良いでしょう。

#### 3. Condaを使用しPythonとライブラリをインストールする

Condaで仮想環境を作成しましょう。Terminalを起動して、次のコマンドを実行してください。今回のハンズオンでは`pandas`という表形式データを扱うためのライブラリと`scikit-learn`という機械学習モデルが簡単に構築できるライブラリを主に扱います。

```
conda create -n ml-handson python pandas scikit-learn matplotlib ipython ipykernel jinja2
```

*ハンズオン資料作成時点では`matplotlib`がPython 3.10で動かないため3.9を指定しています。Python 3.9に特別の依存があるわけではありません*

実行が完了したら、作成した`ml-handson`の環境を有効にするためのコマンドを実行します。

```
conda activate ml-handson
```

Jupyter Notebook上で`ml-handson`が利用できるようにコマンドを実行します。Jupyter Notebook上のコードの実行に使用する仮想環境を「カーネル」と呼び、次のコマンドでは作成した仮想環境をカーネルとして登録しています。

```
ipython kernel install --user --name ml-handson
```

完了したら、Jupyter Notebookの右上にある虫の隣のボタンをクリックしカーネルを切り替えます(作成した仮想環境が反映されるまでに少し時間がかかります)。

![00_environment_setup_005.PNG](images/00_environment_setup_005.PNG)

※ここから先のプログラミング演習では、実行前にカーネルがが`ml-handson`になっていることを確認してください。`default`のままだとインストールされたライブラリが利用できないためです。

#### 4. 構築手順の共有

環境にインストールしたPythonやライブラリをファイルに書き出すことで、環境構築の手順を記録します。環境を別の端末で作成する際は、書き出したファイルをもとに構築ができます。次のコマンドでファイルへの書き出しができます。一般的にCondaの環境は`environment.yml`に記録します。

In [1]:
!conda env export -n ml-handson > environment.yml

`environment.yml`を開くと、インストールした覚えのないライブラリも記録されていると思います。依存ライブラリが記録されるためです。  
依存ライブラリの記載が不要な場合は、実際に必要なライブラリだけ記載するよう修正します。

In [2]:
from pathlib import Path


print(Path("environment.yml").open().read())

name: ml-handson
channels:
  - defaults
dependencies:
  - asttokens=2.0.5=pyhd3eb1b0_0
  - backcall=0.2.0=pyhd3eb1b0_0
  - blas=1.0=mkl
  - bottleneck=1.3.5=py310h9128911_0
  - brotli=1.0.9=ha925a31_2
  - bzip2=1.0.8=he774522_0
  - ca-certificates=2022.4.26=haa95532_0
  - certifi=2022.6.15=py310haa95532_0
  - colorama=0.4.5=py310haa95532_0
  - cycler=0.11.0=pyhd3eb1b0_0
  - debugpy=1.5.1=py310hd77b12b_0
  - decorator=5.1.1=pyhd3eb1b0_0
  - entrypoints=0.4=py310haa95532_0
  - executing=0.8.3=pyhd3eb1b0_0
  - fonttools=4.25.0=pyhd3eb1b0_0
  - freetype=2.10.4=hd328e21_0
  - icc_rt=2019.0.0=h0cc432a_1
  - icu=58.2=ha925a31_3
  - intel-openmp=2021.4.0=haa95532_3556
  - ipykernel=6.9.1=py310haa95532_0
  - ipython=8.3.0=py310haa95532_0
  - jedi=0.18.1=py310haa95532_1
  - jinja2=3.0.3=pyhd3eb1b0_0
  - joblib=1.1.0=pyhd3eb1b0_0
  - jpeg=9e=h2bbff1b_0
  - jupyter_client=7.2.2=py310haa95532_0
  - jupyter_core=4.10.0=py310haa95532_0
  - kiwisolver=1.4.2=py310hd77b12b_0
  - libffi=3.4.2=hd77b12b_4


#### Extra Tutorial

#### Extra-1. Jupyter Lab

[Jupyter Lab](https://jupyter.org/)はソースコードと実行結果を含むドキュメントを作成できるJupyter Notebookの高機能版です。オープンソースで開発されています。Jupyter Notebookではセルと呼ばれる単位で文書を作成していきます。実行済みのセルで作成した変数はそれより下の他のセルからも参照可能です。

* セルの追加
  * 上に追加: A、下に追加: B
* セルの削除
  * DD
* セルの種別の変更
  * ソースコード: Y
  * 文書: M
* セルの実行
  * 実行: Ctrl + Enter
  * 実行して下に移動: Shift + Enter

#### Extra-2. Python Tutorial

今後のプログラミング演習ではPythonを使用します。Pythonの文法はここでは詳しく触れませんので、Referenceの資料を参照してください。各大学から素晴らしい教材が公開されています。次に簡単な計算例を示します。

In [3]:
x = 2
y = 2
print(x + y)

4



### コミュニケーション演習

環境構築として次の手順を実施しました。

1. SageMaker Studio Labを起動
2. `git clone`でファイルを取得
3. Condaを使用しPythonとライブラリをインストールする
4. インストールしたライブラリを記録したファイルを作成する

作業を行う際、誰にどのような依頼をすべきでしょうか。

ロールの種別

| 英語名             | 日本語名                   | 役割                                                                 |
|--------------------|----------------------------|----------------------------------------------------------------------|
| Product Manager    | プロダクトマネージャー     | ユーザー体験を向上させるために実装すべきソフトウェア機能を定義する。 |
| Business Analyst   | ビジネスアナリスト         | 解決すべきビジネス上の問題を定義する                                 |
| Data Analyst       | データアナリスト           | データの可視化と分析で問題を定量的に特定する。                       |
| Architect          | アーキテクト               | ソフトウェアアーキテクチャ全体を設計する。                           |
| DevOps Engineer    | DevOpsエンジニア           | ソフトウェアの開発・運用プロセスを自動化する。                       |
| Software Engineer  | ソフトウェアエンジニア     | ソフトウェアの開発を行う。                                           |
| Operator           | 業務担当者                 | ソフトウェアを利用し業務を行う。                                     |
| System Admin       | システム管理者             | サービスの挙動を監視する。                                           |
| IT Auditor         | IT監督者                   | システムや会社全体の権限管理や監査を行う。                           |
| Data architect     | データアーキテクト         | データを管理する基盤を設計、運用する。                               |
| Domain Expert      | 業務有識者                 | 深い業務知識を持ちデータの意味やあるべき状態について定義する。       |
| Data Engineer      | データエンジニア           | 機械学習モデルに入力可能なデータと特徴を作成する。                   |
| Data Scientist     | データサイエンティスト     | 機械学習モデルを開発する。                                           |
| ML Engineer        | 機械学習エンジニア         | 機械学習モデルを本番環境にデプロイ可能な形式に変換する。             |
| ML Operator        | 業務担当者(機械学習側)     | 推論結果に基づき業務を行いつつ、フィードバックを与える。             |
| Model risk Manager | 機械学習モデルリスク管理者 | 機械学習モデルの推論の挙動を監視する。                               |
| MLOps Engineer     | MLOpsエンジニア            | 機械学習モデルの開発・運用プロセスを自動化する。                     |
| AI/ML Architect    | AI/MLアーキテクト          | 機械学習モデル開発に必要なアーキテクチャ全体を設計する。             |

依頼の種別

* 情報共有: 開発に必要な情報/ドキュメントの提供を依頼する
* 作業: 開発に必要な作業を分担(依頼)する
* レビュー: 作業の確度を上げるため/承認を得るためレビューを依頼する

![communication](images/communication.PNG)


#### DevOpsチームへの依頼例

DevOps EngineerやArchitectに対して次の依頼をするとよいでしょう。

* 情報共有の依頼
   * ソースコード管理にどんなサービスを利用しているか。ブランチ運用ルールはあるか。
   * 開発環境として利用してよい環境はあるか(AWSアカウントなど)。
   * 現在の開発でPythonは利用しているか。利用しているPythonのバージョンはなにか。
   * コーディングルールなどの開発標準があるか。
* 作業依頼
   * 開発環境にアクセスするためのIDの払い出し。
* レビュー
   * 作成した環境構築手順書のレビュー。

#### システム管理チームへの依頼例
IT Auditorに対して次の依頼をするとよいでしょう。

* 情報共有の依頼
   * インストール可能なソフトウェアの制限。
   * プロキシなどネットワークアクセスの制限。
   * 環境構築を行う上で制限を解除する必要がある場合の申請方法/申請ルート。
* 作業依頼
   * 開発端末の調達。
   * (制限の解除)。

## AWS Empowerment

AWSでは、機械学習を行うための環境をマネージドサービスを提供しています。サービスを利用していただくことで、効率的な評価が可能になります。

* [Amazon SageMaker Studio Lab](https://aws.amazon.com/jp/sagemaker/studio-lab/)
* [Amazon SageMaker](https://aws.amazon.com/jp/sagemaker/)
   * データサイエンティスト向けの統合開発環境です。SageMaker Studioを通じ、モデルの学習はもちろん前処理や特徴エンジニアリング、推論エンドポイントの作成まで一気通貫で行うことができます。

## Next Step

[Business Understanding](01_business_understanding.ipynb)


## References

1. Anaconda. [Terms of Service](https://www.anaconda.com/terms-of-service)
2. Greg Markovich. [Miniconda vs Anaconda | The Similarities and Differences](https://linuxnetmag.com/miniconda-vs-anaconda/). 2020.
3. [What is the difference between pip and conda?](https://stackoverflow.com/questions/20994716/what-is-the-difference-between-pip-and-conda)
4. ymotongpoo. [wheelのありがたさとAnacondaへの要望](https://ymotongpoo.hatenablog.com/entry/2017/02/02/182647). 2017.
5. [Misleading description of poetry](https://github.com/pypa/packaging.python.org/issues/506)
6. vaaaaanquish. [pipとpipenvとpoetryの技術的・歴史的背景とその展望](https://vaaaaaanquish.hatenablog.com/entry/2021/03/29/221715). 2021.
7. 宇賀神みずき. [データサイエンティストのための Git 入門](https://aws.amazon.com/jp/builders-flash/202207/git-introduction-for-data-schientist/?awsf.filter-name=*all). 2022.
8. Pythonの教材
   * 岡崎直観. [Python早見帳](https://chokkan.github.io/python/index.html). 2020.
   * 三谷純. [Python ゼロからはじめるプログラミング](https://mitani.cs.tsukuba.ac.jp/book_support/python/). 2021.
   * 東京大学 数理・情報教育研究センター. [Pythonプログラミング入門](https://utokyo-ipp.github.io/). 2020.
